In [120]:
import numpy as np
import pandas as pd
import csv
import pandas as pd
from datetime import datetime
import glob

In [121]:
df1 = pd.read_csv('resale-flat-prices-based-on-approval-date-1990-1999.csv')
df2 = pd.read_csv('resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
df3 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2015-onwards.csv')
df4 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')

In [122]:
df3.drop(columns=['remaining_lease'], inplace=True)

In [123]:
df_all = pd.concat([pd.DataFrame(df1),pd.DataFrame(df2), pd.DataFrame(df3), pd.DataFrame(df4)]).reset_index(drop=True)

In [124]:
df_all['month'] =  pd.to_datetime(df_all['month'], format='%Y-%m')

In [125]:
df_all['lease_commence_date'] =  pd.to_datetime(df_all['lease_commence_date'], format='%Y')

In [126]:
df_all['year'] = pd.DatetimeIndex(df_all['month']).year

In [127]:
df_all['lease_commence_year'] = pd.DatetimeIndex(df_all['lease_commence_date']).year

In [128]:
features = pd.get_dummies(df_all[['flat_model','resale_price', 'floor_area_sqm', 'lease_commence_year', 'year', 'storey_range','street_name', 'town']])

In [129]:
label = df_all[['flat_type', 'year']]

In [33]:
#df_test = df_all[df_all['year']==2015]

In [34]:
#df_train = df_all[(df_all['year']==2014) |(df_all['year']==2013) |(df_all['year']==2012) |(df_all['year']==2011)|(df_all['year']==2010)]

In [130]:
df_test_x =features[(features['year']==2015)]
df_test_y = label [(label['year']==2015)]
df_test_y = df_test_y.drop(['year'], axis=1)

In [131]:
df_train_x =features[(features['year']==2014)| (features['year']==2013)|(features['year']==2012)|(features['year']==2011)|(features['year']==2010)]
df_train_y = label [(label['year']==2014) | (label['year']==2013) | (label['year']==2012) | (label['year']==2011)| (label['year']==2010)]
df_train_y = df_train_y.drop(['year'], axis=1)

In [132]:
from sklearn.ensemble import RandomForestClassifier

In [133]:
clf=RandomForestClassifier(n_estimators=100)

In [134]:
clf.fit(df_train_x,df_train_y )

C:\Users\tehs2\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [135]:
y_pred=clf.predict(df_test_x )

In [136]:
y_pred

array(['3 ROOM', '3 ROOM', '3 ROOM', ..., 'EXECUTIVE', 'EXECUTIVE',
       'EXECUTIVE'], dtype=object)

In [148]:
temp1 = pd.DataFrame(y_pred.reshape(len(y_pred), -1)).reset_index(drop=True)

In [151]:
temp2 = df_test_y.reset_index(drop=True)

In [161]:
temp1.to_csv('classification1.csv', encoding='utf-8-sig')

In [162]:
temp2.to_csv('classification2.csv', encoding='utf-8-sig')

In [67]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(df_test_y, y_pred))

Accuracy: 0.9961754780652419


In [94]:
df_train_y = df_train_y.drop(['weight'], axis=1)

In [95]:
set(df_train_y['flat_type'])

{'1 ROOM',
 '2 ROOM',
 '3 ROOM',
 '4 ROOM',
 '5 ROOM',
 'EXECUTIVE',
 'MULTI-GENERATION'}

In [115]:
class_weight = {'1 ROOM': 0.1,
                '2 ROOM': 1,
                '3 ROOM': 0.1,
               '4 ROOM': 0.1,
               '5 ROOM': 0.1,
               'EXECUTIVE': 0.1,
              'MULTI-GENERATION': 0.1 }

In [116]:
alf=RandomForestClassifier(n_estimators=100, class_weight=class_weight)

In [117]:
alf.fit(df_train_x,df_train_y )

C:\Users\tehs2\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True,
            class_weight={'1 ROOM': 0.1, '2 ROOM': 1, '3 ROOM': 0.1, '4 ROOM': 0.1, '5 ROOM': 0.1, 'EXECUTIVE': 0.1, 'MULTI-GENERATION': 0.1},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [118]:
y_pred==alf.predict(df_test_x )

array([ True,  True,  True, ...,  True,  True,  True])

In [119]:
print("Accuracy:",metrics.accuracy_score(df_test_y, y_pred))

Accuracy: 0.9961754780652419
